In [ ]:
# Import python packages
import pandas as pd
import numpy as np
import streamlit as st
import altair as alt
from snowflake.cortex import Summarize
from snowflake.snowpark.functions import col

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT * FROM DISPUTE_ANALYSIS;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("STATUS OF DISPUTE FOR EACH PRODUCT_TYPE WITH AMOUNT ❄️")
st.bar_chart(my_df, x='PRODUCT_TYPE', y='TRANSACTION_AMOUNT', color='STATUS')



In [ ]:
CREATE OR REPLACE VIEW DISPUTE_SENTIMENT AS
SELECT
    *,
    SNOWFLAKE.CORTEX.SENTIMENT(CUSTOMER_STATEMENT) AS SENTIMENT_RESULT
FROM
    DISPUTE_ANALYSIS
WHERE STATUS='REJECTED';


In [ ]:
select * from dispute_sentiment where sentiment_result<-0.5;

In [ ]:
df= cell5.to_pandas()
results_df=pd.DataFrame(df)
st.header("Need Summary?")
selected_indices = st.selectbox("What summary do you want?", results_df.index)

selected_ted= results_df.loc[selected_indices]
st.text("Selected Customer Review:")
st.dataframe(selected_ted)

talk_content= session.table("dispute_sentiment").select('CUSTOMER_STATEMENT').filter(col('DISPUTE_ID')==selected_ted.DISPUTE_ID).collect()
text_to_summarize=talk_content[0].CUSTOMER_STATEMENT
text_summarise= Summarize(text_to_summarize)

st.title("Summary")
st.markdown(text_summarise)